<a href="https://colab.research.google.com/github/MariaInyutina/Parcing/blob/main/Parcing_extra_TG_weekly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install telethon

In [ ]:
from telethon import TelegramClient
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.tl.types import MessageMediaPhoto
from datetime import datetime, timezone, timedelta
import pandas as pd

# Параметры для Telegram API
API_ID = 20429199
API_HASH = 'e041ef27cceabaece9e91f56e9f0ae4d'
CHANNEL_USERNAME = 'extrahse'  # Имя пользователя канала
phone = '79165879306'
session_name = 'parcing'

# Инициализация клиента
client = TelegramClient(session_name, API_ID, API_HASH)

# Функция для сбора статистики за определённый период
async def get_channel_stats(start_date, end_date):
    await client.start()
    stats = []
    offset_id = 0  # Начальный offset_id для получения сообщений
    limit = 100  # Лимит сообщений за один запрос
    unique_messages = set()  # Множество для отслеживания уникальных идентификаторов сообщений

    # Добавляем один день к end_date, чтобы захватить все сообщения до конца этого дня
    end_date += timedelta(days=1)

    while True:
        # Получаем историю сообщений с использованием offset_id
        history = await client(GetHistoryRequest(
            peer=CHANNEL_USERNAME,
            offset_id=offset_id,
            offset_date=None,
            add_offset=0,
            limit=limit,
            max_id=0,
            min_id=0,
            hash=0
        ))

        if not history.messages:
            print("Достигнут конец истории чата.")
            break  # Выходим, если сообщений больше нет

        for message in history.messages:
            # Проверяем, что сообщение входит в нужный диапазон дат
            if message.date < start_date:
                print("Достигли сообщений старше начальной даты.")
                break
            if message.date >= end_date:
                continue

            # Проверяем уникальность сообщения по ссылке
            post_link = f'https://t.me/{CHANNEL_USERNAME}/{message.id}'
            if post_link in unique_messages:
                continue  # Пропускаем, если это сообщение уже обработано

            # Добавляем ссылку сообщения в набор уникальных сообщений
            unique_messages.add(post_link)

            # Определение заголовка в зависимости от типа контента
            if message.message:
                post_title = message.message[:30]  # Первые 30 символов текста
            elif isinstance(message.media, MessageMediaPhoto):
                post_title = "Фото"  # Отметка "Фото" для изображений
            else:
                post_title = "Без текста"

            # Подсчет количества реакций
            reactions_count = sum(reaction.count for reaction in message.reactions.results) if message.reactions else 0

            # Устанавливаем публичные и непубличные репосты как "-"
            public_reposts = "-"
            private_reposts = "-"

            # Сбор информации о сообщении
            data = {
                'date': message.date.strftime('%d-%m-%y'),  # Форматируем дату как ДД-ММ-ГГ
                'post_title': post_title,  # Используем определенный тип сообщения в качестве заголовка
                'post_link': post_link,  # Ссылка на пост
                'views': message.views,
                'reactions': reactions_count,
                'comments': message.replies.replies if message.replies else 0,
                'public_reposts': public_reposts,  # Заменено на "-"
                'private_reposts': private_reposts,  # Заменено на "-"
            }
            stats.append(data)

        # Обновляем offset_id для следующей итерации
        offset_id = history.messages[-1].id
        print(f"Обработано сообщений: {len(stats)}, Текущий offset_id: {offset_id}")

        # Прерываем, если достигли сообщений раньше указанного start_date
        if history.messages[-1].date < start_date:
            break

    # Создаем DataFrame и сохраняем в Excel
    df = pd.DataFrame(stats)
    df.to_excel('telegram_extra_stats.xlsx', index=False)  # Сохранение в файл Excel
    print("Данные успешно сохранены в telegram_extra_stats.xlsx")

# Запрос ввода даты от пользователя
start_date_input = input("Введите начальную дату в формате ГГГГ-ММ-ДД: ")
end_date_input = input("Введите конечную дату в формате ГГГГ-ММ-ДД: ")

# Преобразование ввода в формат даты с временной зоной UTC
start_date = datetime.strptime(start_date_input, '%Y-%m-%d').replace(tzinfo=timezone.utc)
end_date = datetime.strptime(end_date_input, '%Y-%m-%d').replace(tzinfo=timezone.utc)

# Основная функция с использованием async with
async def main():
    async with client:
        await get_channel_stats(start_date, end_date)

# Запуск main() в асинхронной среде
await main()


Введите начальную дату в формате ГГГГ-ММ-ДД: 2024-11-04
Введите конечную дату в формате ГГГГ-ММ-ДД: 2024-11-08
Достигли сообщений старше начальной даты.
Обработано сообщений: 34, Текущий offset_id: 2785
Данные успешно сохранены в telegram_extra_stats.xlsx


**Использовать следующую ячейку при многоразовом запуске кода, нужна для завершения сеанса**

In [ ]:
# Завершение клиента и пауза
await client.disconnect()
#await asyncio.sleep(1)  # Небольшая пауза для завершения фоновых задач

**Посмотреть получившуюсь таблицу**

In [ ]:
df = pd.read_excel('telegram_extra_stats.xlsx')
df

,date,post_title,post_link,views,reactions,comments,public_reposts,private_reposts
0,08-11-24,Фото,https://t.me/extrahse/2888,2011,0,0,-,-
1,08-11-24,Фото,https://t.me/extrahse/2887,2021,0,0,-,-
2,08-11-24,Фото,https://t.me/extrahse/2886,2010,0,0,-,-
3,08-11-24,Фото,https://t.me/extrahse/2885,1928,0,0,-,-
4,08-11-24,Фото,https://t.me/extrahse/2884,1941,0,0,-,-
5,08-11-24,Фото,https://t.me/extrahse/2883,1953,0,0,-,-
6,08-11-24,Фото,https://t.me/extrahse/2882,1869,0,0,-,-
7,08-11-24,Фото,https://t.me/extrahse/2881,1867,0,0,-,-
8,08-11-24,Фото,https://t.me/extrahse/2880,1877,0,0,-,-
9,08-11-24,Твой настоящий red green flag,https://t.me/extrahse/2879,1843,43,3,-,-


In [ ]:
df = df[~df['post_title'].str.contains("Фото")]
df

,date,post_title,post_link,views,reactions,comments,public_reposts,private_reposts
9,08-11-24,Твой настоящий red green flag,https://t.me/extrahse/2879,1840,43,3,20,NaN
19,08-11-24,"Каждое поколение считает, что",https://t.me/extrahse/2869,2248,36,3,32,NaN
20,06-11-24,"Бу! Испугались? 👻\n\nНе бойтесь,",https://t.me/extrahse/2868,2268,45,6,8,NaN
21,05-11-24,Танцевальный SOS ⚡️\n\nВесь нояб,https://t.me/extrahse/2867,2620,56,6,60,NaN
22,05-11-24,Без текста,https://t.me/extrahse/2866,2327,28,0,1,NaN
23,05-11-24,Всем привет! Всем хорошей прод,https://t.me/extrahse/2865,2216,39,21,0,NaN
33,04-11-24,С Днем народного единства 💔\n\nС,https://t.me/extrahse/2853,2929,77,0,20,NaN


# **Для ежемесячной статистики**:

**Скачиваем финальную таблицу**

In [ ]:
df_transposed.to_excel('telegram_extra_stats.xlsx', index=True)
from google.colab import files

# Скачивание файла
files.download('telegram_extra_stats.xlsx')

# **Для еженедельной статистики**

In [ ]:
# Преобразуем столбец 'date' в формат datetime для сортировки
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%y')

# Сортируем DataFrame по возрастанию даты
df = df.sort_values(by='date').reset_index(drop=True)

# Преобразуем столбец с датой обратно в формат ДД-ММ после сортировки
df['date'] = df['date'].dt.strftime('%d-%m')

# Переставляем столбцы в нужном порядке
df = df[['post_title', 'post_link', 'date', 'views', 'reactions', 'public_reposts', 'private_reposts', 'comments']]

# Транспонируем DataFrame, чтобы сделать столбцы строками
df_transposed = df.T

# Переименовываем индексы в соответствии с требуемыми названиями
df_transposed.index = [
    'Название поста',
    'Ссылка',
    'Дата',
    'Просмотры',
    'Реакции',
    'Репосты публичные',
    'Репосты непубличные',
    'Комментарии'
]

# Теперь df_transposed содержит нужную структуру с датами в порядке возрастания
df_transposed


<ipython-input-48-12a9488c7192>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'], format='%d-%m-%y')


,0,1,2,3,4,5,6
Название поста,С Днем народного единства 💔\n\nС,Танцевальный SOS ⚡️\n\nВесь нояб,Без текста,Всем привет! Всем хорошей прод,"Бу! Испугались? 👻\n\nНе бойтесь,",Твой настоящий red green flag,"Каждое поколение считает, что"
Ссылка,https://t.me/extrahse/2853,https://t.me/extrahse/2867,https://t.me/extrahse/2866,https://t.me/extrahse/2865,https://t.me/extrahse/2868,https://t.me/extrahse/2879,https://t.me/extrahse/2869
Дата,04-11,05-11,05-11,05-11,06-11,08-11,08-11
Просмотры,2929,2620,2327,2216,2268,1840,2248
Реакции,77,56,28,39,45,43,36
Репосты публичные,20,60,1,0,8,20,32
Репосты непубличные,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Комментарии,0,6,0,21,6,3,3


**Скачиваем финальную таблицу**

In [ ]:
df_transposed.to_excel('telegram_extra_stats.xlsx', index=True)
from google.colab import files

# Скачивание файла
files.download('telegram_extra_stats.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>